In [0]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
#from sklearn.ensemble import AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
train_df = pd.read_json("train.json")
test_df = pd.read_json("test.json")

In [2]:
# train_df["created"] = pd.to_datetime(train_df["created"])
# train_df["created_year"] = train_df["created"].dt.year
# train_df["created_month"] = train_df["created"].dt.month
# train_df["created_day"] = train_df["created"].dt.day
# train_df["created_hour"] = train_df["created"].dt.hour
# train_df["num_features"] = train_df["features"].apply(len)
# train_df["num_photos"] = train_df["photos"].apply(len)
train_df.drop(columns=['created', 'display_address', 'description', 'street_address', 'photos'], inplace=True)
train_df.drop(columns=['building_id', 'manager_id', 'listing_id'], inplace=True)
train_df.drop(columns=['features'], inplace=True)
train_df

,bathrooms,bedrooms,latitude,longitude,price,interest_level
4,1.0,1,40.7108,-73.9539,2400,medium
6,1.0,2,40.7513,-73.9722,3800,low
9,1.0,2,40.7575,-73.9625,3495,medium
10,1.5,3,40.7145,-73.9425,3000,medium
15,1.0,0,40.7439,-73.9743,2795,low
...,...,...,...,...,...,...
124000,1.0,3,40.8433,-73.9396,2800,low
124002,1.0,2,40.8198,-73.9578,2395,medium
124004,1.0,1,40.5765,-73.9554,1850,medium
124008,1.0,2,40.7448,-74.0017,4195,medium


In [3]:
test_df2 = test_df.copy()
# test_df2["created"] = pd.to_datetime(test_df2["created"])
# test_df2["created_year"] = test_df2["created"].dt.year
# test_df2["created_month"] = test_df2["created"].dt.month
# test_df2["created_day"] = test_df2["created"].dt.day
# test_df2["created_hour"] = test_df2["created"].dt.hour
# test_df2["num_features"] = test_df2["features"].apply(len)
# test_df2["num_photos"] = test_df2["photos"].apply(len)
test_df2.drop(columns=['created', 'display_address', 'description', 'street_address', 'photos'], inplace=True)
test_df2.drop(columns=['building_id', 'manager_id', 'listing_id'], inplace=True)
test_df2.drop(columns=['features'], inplace=True)
test_df2

,bathrooms,bedrooms,latitude,longitude,price
0,1.0,1,40.7185,-73.9865,2950
1,1.0,2,40.7278,-74.0000,2850
2,1.0,0,40.7260,-74.0026,2295
3,1.0,2,40.7321,-74.0028,2900
5,1.0,1,40.7054,-74.0095,3254
...,...,...,...,...,...
124003,1.0,1,40.7925,-73.9454,1700
124005,1.0,2,40.7456,-73.9797,4195
124006,1.0,0,40.7416,-73.9829,2400
124007,2.0,2,40.7485,-73.9800,6895


In [0]:
target_map = {'high':0, 'medium':1, 'low':2}
X = train_df[train_df.columns.difference(['interest_level'])]
y = np.array(train_df['interest_level'].apply(lambda x: target_map[x]))
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)
kfold = model_selection.KFold(n_splits=5)
# ada = AdaBoostClassifier(n_estimators=100, random_state=0, learning_rate=0.1)
# ada.fit(X_train, y_train)
# y_val_pred = ada.predict_proba(X_val)
# y_val_pred_acc = ada.predict(X_val)
knn = KNeighborsClassifier(n_neighbors=275)
knn.fit(X_train, y_train)
y_val_pred = knn.predict_proba(X_val)
y_val_pred_acc = knn.predict(X_val)

In [5]:
results = model_selection.cross_val_score(knn, X, y, cv=kfold)
results.mean()

0.6980870350110139

In [6]:
logloss = log_loss(y_val, y_val_pred)
logloss

0.743334004121507

In [7]:
accuracy = metrics.accuracy_score(y_val, y_val_pred_acc)
accuracy

0.696936206790692

In [8]:
print(classification_report(y_val, y_val_pred_acc, target_names=['high', 'medium','low']))

              precision    recall  f1-score   support

        high       0.54      0.09      0.15      1299
      medium       0.00      0.00      0.00      3704
         low       0.70      1.00      0.82     11284

    accuracy                           0.70     16287
   macro avg       0.41      0.36      0.32     16287
weighted avg       0.53      0.70      0.58     16287



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [0]:
X_test = test_df2
y_test = knn.predict_proba(X_test)
sub = pd.DataFrame()
sub['listing_id'] = test_df['listing_id']
for label in ['high', 'medium', 'low']:
  sub[label] = y_test[:, target_map[label]]
sub.to_csv("submission.csv", index=False)